In [1]:
import pandas as pd
data = pd.read_csv('data/dataPr/steam_games_ek_20240613.csv')
data.shape

(42497, 35)

In [2]:
print(data.columns)
data.head()

Index(['app_id', 'title', 'release_date', 'genres', 'categories', 'developer',
       'publisher', 'original_price', 'discount_percentage',
       'discounted_price', 'dlc_available', 'age_rating', 'content_descriptor',
       'about_description', 'win_support', 'mac_support', 'linux_support',
       'awards', 'overall_review', 'overall_review_%', 'overall_review_count',
       'recent_review', 'recent_review_%', 'recent_review_count', 'title_len',
       'genre_num', 'cat_num', 'self_pub', 'cont_desc_len', 'abt_desc_length',
       'windows', 'mac', 'linux', 'dlc', 'year_month'],
      dtype='object')


,app_id,title,release_date,genres,categories,developer,publisher,original_price,discount_percentage,discounted_price,...,genre_num,cat_num,self_pub,cont_desc_len,abt_desc_length,windows,mac,linux,dlc,year_month
0,730,Counter-Strike 2,2012-08-21,"Action, Free to Play","Cross-Platform Multiplayer, Steam Trading Card...",Valve,Valve,NaN,NaN,NaN,...,2,9,True,0,3,1,0,1,1,2012-08
1,570,Dota 2,2013-07-09,"Action, Strategy, Free to Play","Steam Trading Cards, Steam Workshop, SteamVR C...",Valve,Valve,NaN,NaN,NaN,...,3,5,True,0,7,1,1,1,1,2013-07
2,2215430,Ghost of Tsushima DIRECTOR'S CUT,2024-05-16,"Action, Adventure","Single-player, Online Co-op, Steam Achievement...",Sucker Punch Productions,PlayStation PC LLC,NaN,NaN,47.988,...,2,4,False,1,2,1,0,0,0,2024-05
3,1245620,ELDEN RING,2022-02-24,"Action, RPG","Single-player, Online PvP, Online Co-op, Steam...",FromSoftware Inc.,FromSoftware Inc.,NaN,NaN,43.188,...,2,7,True,1,3,1,0,0,1,2022-02
4,1085660,Destiny 2,2019-10-01,"Action, Adventure, Free to Play","Single-player, Online PvP, Online Co-op, Steam...",Bungie,Bungie,NaN,NaN,NaN,...,3,5,False,2,3,1,0,0,1,2019-10


In [3]:
columns = ["recent_review_%", "recent_review_count", "title_len", "genre_num", "cat_num" ,"self_pub", "windows", "mac", "linux", "about_description"]
sel = data[columns].copy()
sel.dropna(inplace=True)
sel.shape
sel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5503 entries, 0 to 39143
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   recent_review_%      5503 non-null   float64
 1   recent_review_count  5503 non-null   float64
 2   title_len            5503 non-null   int64  
 3   genre_num            5503 non-null   int64  
 4   cat_num              5503 non-null   int64  
 5   self_pub             5503 non-null   bool   
 6   windows              5503 non-null   int64  
 7   mac                  5503 non-null   int64  
 8   linux                5503 non-null   int64  
 9   about_description    5503 non-null   object 
dtypes: bool(1), float64(2), int64(6), object(1)
memory usage: 435.3+ KB


In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

lc_embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
embed_model = LangchainEmbedding(lc_embed_model)


/home/leopach/PycharmProjects/DSS/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/leopach/PycharmProjects/DSS/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-01 10:21:33.444856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been re

In [5]:
def get_emm(text):
    embeddings = embed_model.get_text_embedding(
    text
)
    return (len(embeddings), embeddings[:10])

descriptions = list(sel["about_description"])
embeddings = [get_emm(descriptions[k])[1] for k in range(len(descriptions))]

    

In [6]:
for k in range(10):
    sel[f"emb_{k}"] = [vec[k] for vec in embeddings]
sel.describe()

,recent_review_%,recent_review_count,title_len,genre_num,cat_num,windows,mac,linux,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9
count,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.0,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000,5503.000000
mean,82.303107,282.100491,18.862439,2.839906,5.076685,1.0,0.279120,0.180447,0.007626,0.017989,-0.005733,0.013818,-0.016223,0.006319,-0.024226,0.002931,-0.019475,0.013014
std,15.110260,4583.606042,10.438712,1.413954,2.383514,0.0,0.448608,0.384594,0.031437,0.042943,0.018755,0.030840,0.031800,0.024171,0.036499,0.024643,0.042443,0.026895
min,6.000000,10.000000,1.000000,0.000000,0.000000,1.0,0.000000,0.000000,-0.112446,-0.134833,-0.074660,-0.098015,-0.136190,-0.081351,-0.166426,-0.090666,-0.185090,-0.081787
25%,75.000000,16.000000,11.000000,2.000000,3.000000,1.0,0.000000,0.000000,-0.012961,-0.010326,-0.018462,-0.006875,-0.037613,-0.010340,-0.048883,-0.013673,-0.047316,-0.004767
50%,86.000000,34.000000,16.000000,3.000000,5.000000,1.0,0.000000,0.000000,0.008112,0.018612,-0.005508,0.014241,-0.016613,0.006482,-0.024755,0.003037,-0.019389,0.012920
75%,93.000000,102.000000,25.000000,4.000000,6.000000,1.0,1.000000,0.000000,0.028529,0.047623,0.007189,0.035034,0.004881,0.022405,-0.000263,0.018786,0.009040,0.030955
max,100.000000,328633.000000,83.000000,9.000000,18.000000,1.0,1.000000,1.000000,0.115968,0.164334,0.063171,0.119425,0.104926,0.087821,0.088789,0.101171,0.174025,0.102607


In [7]:
sel.drop("about_description", axis=1, inplace=True)

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Assume 'data' is your DataFrame and already loaded

# Define the target variable and features
X = sel.drop(columns=['recent_review_%'])
y = sel['recent_review_%']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost regressor
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                          max_depth=5, alpha=10, n_estimators=10)

# Fit the model to the training data
xg_reg.fit(X_train, y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, ...)

In [9]:
from sklearn.metrics import mean_squared_error

# Make predictions on the test set
y_pred = xg_reg.predict(X_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 235.24913429082537


In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'colsample_bytree': [0.3, 0.7]
}

grid_search = GridSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror'),
                           param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)

grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")

Fitting 3 folds for each of 54 candidates, totalling 162 fits
Best parameters: {'colsample_bytree': 0.3, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}


In [11]:
# Extract the best model from grid search
best_model = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error of the best model: {mse}")

Mean Squared Error of the best model: 227.86871759267007


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert boolean column to integer
sel['self_pub'] = sel['self_pub'].astype(int)

# Define the target variable and features
X = sel.drop(columns=['recent_review_%'])
y = sel['recent_review_%']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Set TensorFlow to use only the CPU
tf.config.set_visible_devices([], 'GPU')

# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))  # Output layer for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
test_loss, test_mse = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Mean Squared Error: {test_mse}")


/home/leopach/PycharmProjects/DSS/venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1722522476.575890   55068 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722522476.577952   55068 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722522476.578100   55068 cud

Epoch 1/100


I0000 00:00:1722522477.954398   56844 service.cc:146] XLA service 0x7d3d7802f880 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722522477.954432   56844 service.cc:154]   StreamExecutor device (0): Host, Default Version
2024-08-01 10:27:57.982052: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1722522479.145639   56844 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


111/111 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 6403.2261 - mean_squared_error: 6403.2261 - val_loss: 1223.2576 - val_mean_squared_error: 1223.2576
Epoch 2/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 514.1691 - mean_squared_error: 514.1691 - val_loss: 684.2838 - val_mean_squared_error: 684.2838
Epoch 3/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 316.2400 - mean_squared_error: 316.2400 - val_loss: 629.7228 - val_mean_squared_error: 629.7228
Epoch 4/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 304.3615 - mean_squared_error: 304.3615 - val_loss: 586.5250 - val_mean_squared_error: 586.5250
Epoch 5/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 295.6564 - mean_squared_error: 295.6564 - val_loss: 532.2376 - val_mean_squared_error: 532.2376
Epoch 6/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 285.0364 - mean_squared_error: 285.0364 - val_loss: 547.0470 - val_mean_squared_error: 547.0470
Epoch 7/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 274.3

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Assume 'data' is your DataFrame and already loaded

# Convert boolean column to integer
sel['self_pub'] = sel['self_pub'].astype(int)

# Discretize the target variable into 5 classes
sel['recent_review_class'] = pd.cut(sel['recent_review_%'], bins=5, labels=False)

# Define the features and the new target variable
X = sel.drop(columns=['recent_review_%', 'recent_review_class'])
y = sel['recent_review_class']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the base classifier
base_classifier = DecisionTreeClassifier()

# Define the bagging classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=10, random_state=42)

# Train the bagging classifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_classifier.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


/home/leopach/PycharmProjects/DSS/venv/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 0.5667574931880109


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Assume 'data' is your DataFrame and already loaded

# Convert boolean column to integer
sel['self_pub'] = sel['self_pub'].astype(int)

# Discretize the target variable into 5 classes
sel['recent_review_class'] = pd.cut(sel['recent_review_%'], bins=5, labels=False)

# Define the features and the new target variable
X = sel.drop(columns=['recent_review_%', 'recent_review_class'])
y = sel['recent_review_class']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the target variable to one-hot encoding
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer for multiclass classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy}")


Epoch 1/100


/home/leopach/PycharmProjects/DSS/venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


111/111 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4724 - loss: 1.3358 - val_accuracy: 0.6413 - val_loss: 0.9093
Epoch 2/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6130 - loss: 0.9454 - val_accuracy: 0.6334 - val_loss: 0.9085
Epoch 3/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6075 - loss: 0.9712 - val_accuracy: 0.6368 - val_loss: 0.9013
Epoch 4/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6015 - loss: 0.9443 - val_accuracy: 0.6356 - val_loss: 0.8959
Epoch 5/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6152 - loss: 0.9372 - val_accuracy: 0.6322 - val_loss: 0.8999
Epoch 6/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6196 - loss: 0.9146 - val_accuracy: 0.6402 - val_loss: 0.8987
Epoch 7/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6141 - loss: 0.9191 - val_accuracy: 0.6243 - val_loss: 0.8991
Epoch 8/100
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6238 - loss: 0.8985 - val_accuracy: 0.6288

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Define a function to create the model
def create_model(units_1, units_2, learning_rate):
    model = Sequential()
    model.add(Dense(units_1, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(units_2, activation='relu'))
    model.add(Dense(1))  # Output layer for regression
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

# Define the parameter grid
param_grid = {
    'units_1': [32, 64, 128],
    'units_2': [16, 32, 64],
    'learning_rate': [0.001, 0.01, 0.1]
}

# Prepare training and validation data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Grid search
best_mse = float('inf')
best_params = {}

for units_1 in param_grid['units_1']:
    for units_2 in param_grid['units_2']:
        for learning_rate in param_grid['learning_rate']:
            print(f"Training with units_1={units_1}, units_2={units_2}, learning_rate={learning_rate}")
            model = create_model(units_1, units_2, learning_rate)
            history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=0)
            val_mse = model.evaluate(X_val, y_val, verbose=0)[1]
            print(f"Validation MSE: {val_mse}")
            if val_mse < best_mse:
                best_mse = val_mse
                best_params = {'units_1': units_1, 'units_2': units_2, 'learning_rate': learning_rate}

# Print the best hyperparameters
print(f"Best Validation MSE: {best_mse}")
print(f"Best Hyperparameters: {best_params}")

# Train the best model on the full training set and evaluate on the test set
best_model = create_model(best_params['units_1'], best_params['units_2'], best_params['learning_rate'])
best_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=0)
test_loss, test_mse = best_model.evaluate(X_test, y_test, verbose=1)
print(f"Test Mean Squared Error: {test_mse}")


Training with units_1=32, units_2=16, learning_rate=0.001


/home/leopach/PycharmProjects/DSS/venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Validation MSE: 0.6214673519134521
Training with units_1=32, units_2=16, learning_rate=0.01
Validation MSE: 0.6766289472579956
Training with units_1=32, units_2=16, learning_rate=0.1
Validation MSE: 0.5735418796539307
Training with units_1=32, units_2=32, learning_rate=0.001
Validation MSE: 0.6766762733459473
Training with units_1=32, units_2=32, learning_rate=0.01
Validation MSE: 0.6812847256660461
Training with units_1=32, units_2=32, learning_rate=0.1
Validation MSE: 0.5792275071144104
Training with units_1=32, units_2=64, learning_rate=0.001
Validation MSE: 0.7101249694824219
Training with units_1=32, units_2=64, learning_rate=0.01
Validation MSE: 0.7154772281646729
Training with units_1=32, units_2=64, learning_rate=0.1
Validation MSE: 0.5796041488647461
Training with units_1=64, units_2=16, learning_rate=0.001
Validation MSE: 0.6288617849349976
Training with units_1=64, units_2=16, learning_rate=0.01
Validation MSE: 0.7083317637443542
Training with units_1=64, units_2=16, learnin